<a href="https://colab.research.google.com/github/esonwong/learning-python/blob/main/%E8%87%AA%E5%AE%9A%E4%B9%89%E7%9F%A5%E8%AF%86%E5%BA%93%E8%81%8A%E5%A4%A9%E6%9C%BA%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介
你只需要按照这份教程一步一步操作，即可实现一个「自定义知识库的聊天机器人」。 本教程的使用方法很简单，你只需要点击代码前的播放按钮即可。

如果遇到问题，可以再看看这篇使用教程。


# 导入个人知识库数据

导入的方法有两种：

1. 导入在线数据：导入 Github 数据是个相对简答的方式。如果你是第一次使用，我建议你先用这个方法试试。点击下方代码前的播放按钮，就会运行这段代码。运行完成后，会导入我写的几份 newsletter。如果你也想像我那样导入数据，只需要修改 clone 后面的链接地址即可。
2. 导入离线数据：详情请见[教程]()。

In [ ]:
! git clone https://github.com/thinkingjimmy/Jimmy-Newsletter-Corpus.git

# 安装依赖库

点击下方代码前的播放按钮即可，这一步主要是安装好后续程序依赖的库。

In [ ]:
!pip install gpt-index
!pip install langchain
!pip install transformers

# 定义函数
以下代码定义了构建索引和查询索引所需的函数。

In [ ]:
from gpt_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 300
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("请输入你的提问：")
        response = index.query(query, response_mode="compact")
        display(Markdown(f"Response: <b>{response.response}</b>"))

# 设置 OpenAI API Key

点击代码前的播放按钮，运行代码后，在框内输入你的个人 OpenAI API Key，黏贴完记得按下回车键。如何生成 API Key 请见[教程]()。

In [ ]:
os.environ["OPENAI_API_KEY"] = input("将你的 OpenAI API Key 黏贴到这里：")

# 构建索引
这一步程序会将第一步导入的数据都跑一遍，并使用 OpenAI 的 embedings API。如果第一步你上传了自己的数据，只需要将 ' ' 里的 Jimmy-Newsletter-Corpus 修改为你上传的文件夹名称即可。

**注意：**

- 这一步会耗费你的 OpenAI 的 Credit，1000 个 token 的价格是 $0.02，运行以下代码前需要注意你的账号里是否还有钱。
- 如果你用的 OpenAI 账号是个免费账号，你有可能会遇到频率警告，此时可以等一等再运行下方代码（另外你的导入的知识库数据太多，也会触发）。解除这个限制，最好的方式是在你的 OpenAI 账号的 Billing 页面里绑定信用卡。如何绑卡，需要各位自行搜索。



In [ ]:
construct_index('Jimmy-Newsletter-Corpus')

# 提问

这一步你就可以试试提问了，如果你在第一步导入的是我预设的数据，你可以试试问以下问题：
- Issue 90 主要讲了什么什么内容？
- 推荐一本跟 Issue 90 里提到的书类似的书

如果你导入的是自己的资料，也可以问以下几个类型的问题：
- 总结
- 提问
- 信息提取

关于如何提问，可以看看我的另一份教程:[Learning Prompt](https://learningprompt.wiki/)。

In [ ]:
ask_ai()